In [2]:
import tensorflow
import pandas
import mido

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [20]:
mid = mido.MidiFile('WebsterNight.mid')

def notename(note_num):
    octave = (note_num // 12)
    notes = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']
    note = notes[note_num % 12]
    return note + str(octave)

def ticks_to_measure(midi, ticks): 
    #print(str(midi.ticks_per_beat))
    measure_time = midi.ticks_per_beat * 4 
    return ticks / measure_time 

def dict_to_csv(dict_list, file_name): 
    l = [] 
    for dct in dict_list: 
        df = pandas.DataFrame(dct, [0])
        l.append(df)
    print(l)
    tmp = pandas.concat(l)
    tmp = tmp.reindex(columns = ["note_name", "start_time", "duration", "velocity"])
    tmp.to_csv(file_name + '.csv', index = False)
            
# Eventually this will all be OOP!!!
# Running variables (these will keep track of things)
def midi_to_csv(midi):
    notes = []
    
    queue = {}
    
    for i, track in enumerate(mid.tracks):
        print('Track {}: {}'.format(i, track.name))
        curr_time = 0 
        for msg in track:
            print(msg)
            if msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
                curr_note_name = notename(msg.note) 
                curr_note = queue[curr_note_name]
                #curr_note['duration'] = ticks_to_measure(midi, msg.time)
                curr_note['duration'] = msg.time
                curr_time += msg.time
                notes.append(curr_note)
                queue.pop(curr_note_name)
            elif msg.type == 'note_on': 
                curr_note = {} 
                queue[notename(msg.note)] = curr_note 
                curr_note['note_name'] = notename(msg.note)
                curr_note['velocity'] = msg.velocity 
                curr_time += msg.time 
                #curr_note['start_time'] = ticks_to_measure(midi, curr_time)
                curr_note['start_time'] = curr_time
    file_name = midi.filename[:-4]
    #print(notes)
    dict_to_csv(notes, file_name)

midi_to_csv(mid)
                

Track 0: Melody
<meta message track_name name='Melody' time=0>
<meta message set_tempo tempo=331491 time=0>
<meta message key_signature key='D' time=0>
<meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
note_on channel=0 note=53 velocity=94 time=4656
note_off channel=0 note=53 velocity=94 time=119
note_on channel=0 note=57 velocity=95 time=8
note_off channel=0 note=57 velocity=95 time=557
note_on channel=0 note=59 velocity=96 time=509
note_off channel=0 note=59 velocity=96 time=261
note_on channel=0 note=61 velocity=115 time=14
note_off channel=0 note=61 velocity=115 time=134
note_on channel=0 note=59 velocity=92 time=34
note_off channel=0 note=59 velocity=92 time=126
note_on channel=0 note=57 velocity=109 time=18
note_off channel=0 note=57 velocity=109 time=150
note_on channel=0 note=54 velocity=102 time=23
note_off channel=0 note=54 velocity=102 time=186
note_on channel=0 note=53 velocity=98 time=2
note_off channel=0 note=

[   duration note_name  start_time  velocity
0       119        F4        4656        94,    duration note_name  start_time  velocity
0       557        A4        4783        95,    duration note_name  start_time  velocity
0       261        B4        5849        96,    duration note_name  start_time  velocity
0       134       C#5        6124       115,    duration note_name  start_time  velocity
0       126        B4        6292        92,    duration note_name  start_time  velocity
0       150        A4        6436       109,    duration note_name  start_time  velocity
0       186       F#4        6609       102,    duration note_name  start_time  velocity
0        61        F4        6797        98,    duration note_name  start_time  velocity
0       187       F#4        6867        97,    duration note_name  start_time  velocity
0        56        A4        7763       109,    duration note_name  start_time  velocity
0       100       A#4        7834       112,    duration note_nam